In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 

In [12]:
from sklearn.model_selection  import GridSearchCV
from sklearn.linear_model import LogisticRegression 

param_grid={'penalty':['l1', 'l2', 'elasticnet', None],'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],'dual':[True,False],'C':[10,100,1000,1500,2000]}
     
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)


Fitting 5 folds for each of 240 candidates, totalling 1200 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
850 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 1500, 2000],
                         'dual': [True, False],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [13]:

re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 10, 'dual': False, 'penalty': 'l1', 'solver': 'liblinear'}: 0.8906190214115365


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[74  5]
 [ 8 33]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9481321395492436

In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_dual,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001267,0.001719,0.000000,0.000000,10,True,l1,lbfgs,"{'C': 10, 'dual': True, 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
1,0.011766,0.004893,0.000000,0.000000,10,True,l1,liblinear,"{'C': 10, 'dual': True, 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
2,0.001512,0.000949,0.000000,0.000000,10,True,l1,newton-cg,"{'C': 10, 'dual': True, 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
3,0.001352,0.000853,0.000000,0.000000,10,True,l1,newton-cholesky,"{'C': 10, 'dual': True, 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
4,0.001569,0.001166,0.000000,0.000000,10,True,l1,sag,"{'C': 10, 'dual': True, 'penalty': 'l1', 'solv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.000409,0.000818,0.000000,0.000000,2000,False,None,liblinear,"{'C': 2000, 'dual': False, 'penalty': None, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,71
236,0.009886,0.003189,0.012808,0.004666,2000,False,None,newton-cg,"{'C': 2000, 'dual': False, 'penalty': None, 's...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
237,0.006674,0.004869,0.013184,0.003575,2000,False,None,newton-cholesky,"{'C': 2000, 'dual': False, 'penalty': None, 's...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
238,0.005212,0.000371,0.009727,0.002319,2000,False,None,sag,"{'C': 2000, 'dual': False, 'penalty': None, 's...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
